In [2]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/surya/NLP_Assignment/')

Mounted at /content/drive


In [3]:
# Download the necessary NLTK resources
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [4]:
# Import the libraries
import pandas as pd
import spacy
import nltk
from nltk.tag import StanfordNERTagger
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the dataset
df =  pd.read_csv('NER_Dataset.csv',error_bad_lines=False, engine="python")

# Extract the sentences, words, POS tags, and NER tags from the dataset
sentences = df.groupby("Sentence_ID")["Word"].apply(list).values
words = df["Word"].values
pos_tags = df["POS"].values
ner_tags = df["Tag"].values

# Load the SpaCy model for English
nlp = spacy.load("en_core_web_sm")

# Define a function to map the SpaCy tags to the dataset tags
def map_spacy_tags(spacy_tags):
    # Use the tag mapping dictionary provided
    tag_mapping = {
        "B-art": "ART",
        "B-eve": "EVENT",
        "B-geo": "GPE",
        "B-gpe": "GPE",
        "B-nat": "NORP",
        "B-org": "ORG",
        "B-per": "PERSON",
        "B-tim": "TIME",
        "I-art": "ART",
        "I-eve": "EVENT",
        "I-geo": "GPE",
        "I-gpe": "GPE",
        "I-nat": "NORP",
        "I-org": "ORG",
        "I-per": "PERSON",
        "I-tim": "TIME",
        "O": ""
    }
    # Initialize an empty list to store the mapped tags
    mapped_tags = []
    # Loop through the SpaCy tags
    for tag in spacy_tags:
        # If the tag is in the mapping dictionary, use the corresponding value
        if tag in tag_mapping:
            mapped_tags.append(tag_mapping[tag])
        # Otherwise, use the tag as it is
        else:
            mapped_tags.append(tag)
    # Return the mapped tags as a list
    return mapped_tags

# Define a function to get the POS tags and NER tags from SpaCy
def get_spacy_tags(sentences):
    # Initialize empty lists to store the POS tags and NER tags
    spacy_pos_tags = []
    spacy_ner_tags = []
    # Loop through the sentences
    for sentence in sentences:
        # Convert the sentence to a SpaCy document
        doc = nlp(" ".join(sentence))
        # Loop through the tokens in the document
        for token in doc:
            # Append the POS tag and the NER tag to the respective lists
            spacy_pos_tags.append(token.tag_)
            spacy_ner_tags.append(token.ent_type_)
    # Map the SpaCy NER tags to the dataset tags
    spacy_ner_tags = map_spacy_tags(spacy_ner_tags)
    # Return the POS tags and NER tags as lists
    return spacy_pos_tags, spacy_ner_tags

# Get the POS tags and NER tags from SpaCy
spacy_pos_tags, spacy_ner_tags = get_spacy_tags(sentences)

# Define a function to get the POS tags and NER tags from NLTK
def get_nltk_tags(sentences):
    # Initialize empty lists to store the POS tags and NER tags
    nltk_pos_tags = []
    nltk_ner_tags = []
    # Loop through the sentences
    for sentence in sentences:
        # Get the POS tags from NLTK
        pos_tagged = nltk.pos_tag(sentence)
        # Get the NER tags from NLTK
        ner_tagged = nltk.ne_chunk(pos_tagged)
        # Loop through the tagged tokens
        for token in ner_tagged:
            # If the token is a named entity, use the label as the NER tag
            if hasattr(token, "label"):
                nltk_ner_tags.append(token.label())
            # Otherwise, use "O" as the NER tag
            else:
                nltk_ner_tags.append("O")
            # Append the POS tag to the list
            nltk_pos_tags.append(token[1])
    # Return the POS tags and NER tags as lists
    return nltk_pos_tags, nltk_ner_tags

# Get the POS tags and NER tags from NLTK
nltk_pos_tags, nltk_ner_tags = get_nltk_tags(sentences)




<ipython-input-4-a502eaf27f0f>:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df =  pd.read_csv('NER_Dataset.csv',error_bad_lines=False, engine="python")
Skipping line 6553: unexpected end of data


In [ ]:
# Define the path to the StanfordNER model and the jar file
model_path = "/content/drive/MyDrive/surya/NLP_Assignment/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz"
jar_path = "/content/drive/MyDrive/surya/NLP_Assignment/stanford-ner-2018-10-16/stanford-ner.jar"

# Initialize the StanfordNERTagger
st = StanfordNERTagger(model_path, jar_path)

# Define a function to get the NER tags from StanfordNER
def get_stanford_tags(sentences):
    # Initialize an empty list to store the NER tags
    stanford_ner_tags = []
    # Loop through the sentences
    for sentence in sentences:
        # Get the NER tags from StanfordNER
        ner_tagged = st.tag(sentence)
        # Loop through the tagged tokens
        for token in ner_tagged:
            # Append the NER tag to the list
            stanford_ner_tags.append(token[1])
    # Return the NER tags as a list
    return stanford_ner_tags

# Get the NER tags from StanfordNER
stanford_ner_tags = get_stanford_tags(sentences)

#2.

The text processing pipeline that I adopted is as follows:

First, I loaded the dataset from the CSV file and extracted the sentences, words, POS tags, and NER tags as lists.

Second, I loaded the SpaCy model for English and defined a function to map the SpaCy tags to the dataset tags using the tag mapping dictionary provided.

Third, I defined a function to get the POS tags and NER tags from SpaCy by converting each sentence to a SpaCy document and looping through the tokens in the document. I used the tag_ and ent_type_ attributes of the tokens to get the POS tag and the NER tag respectively. I also mapped the SpaCy NER tags to the dataset tags using the mapping function.

Fourth, I defined a function to get the POS tags and NER tags from NLTK by using the pos_tag and ne_chunk functions from the NLTK library. I looped through the tagged tokens and used the label of the named entities as the NER tag and the second element of the tuple as the POS tag. I used “O” as the NER tag for the tokens that were not named entities.

Fifth, I initialized the StanfordNERTagger with the path to the model and the jar file. I defined a function to get the NER tags from StanfordNER by using the tag method of the tagger. I looped through the tagged tokens and used the second element of the tuple as the NER tag.

#3
I used the following code to evaluate the model predictions using accuracy and confusion matrix (TAG-wise) metrics. I also provided a comparative analysis of the results.

In [ ]:
# Check and adjust the lengths of arrays
min_length = min(len(pos_tags), len(spacy_pos_tags), len(nltk_pos_tags), len(spacy_ner_tags), len(nltk_ner_tags))

# Trim arrays to the minimum length
pos_tags = pos_tags[:min_length]
spacy_pos_tags = spacy_pos_tags[:min_length]
nltk_pos_tags = nltk_pos_tags[:min_length]
#ner_tags = ner_tags[:min_length]
spacy_ner_tags = spacy_ner_tags[:min_length]
nltk_ner_tags = nltk_ner_tags[:min_length]
#stanford_ner_tags = stanford_ner_tags[:min_length]

# Import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Define a function to plot the confusion matrix
def plot_confusion_matrix(cm, labels, title):
    # Convert the confusion matrix to a dataframe
    df_cm = pd.DataFrame(cm, index=labels, columns=labels)
    # Plot the heatmap of the dataframe
    plt.figure(figsize=(10,10))
    sns.heatmap(df_cm, annot=True, fmt="d", cmap="Blues")
    # Add the title and the axis labels
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    # Show the plot
    plt.show()

# Define the list of unique POS tags
pos_labels = list(set(pos_tags))

# Define the list of unique NER tags
ner_labels = list(set(ner_tags))

# Calculate the accuracy scores for POS tagging
spacy_pos_accuracy = accuracy_score(pos_tags, spacy_pos_tags)
nltk_pos_accuracy = accuracy_score(pos_tags, nltk_pos_tags)

# Print the accuracy scores for POS tagging
print("Accuracy scores for POS tagging:")
print(f"SpaCy: {spacy_pos_accuracy:.4f}")
print(f"NLTK: {nltk_pos_accuracy:.4f}")

# Calculate the confusion matrices for POS tagging
spacy_pos_cm = confusion_matrix(pos_tags, spacy_pos_tags, labels=pos_labels)
nltk_pos_cm = confusion_matrix(pos_tags, nltk_pos_tags, labels=pos_labels)

# Plot the confusion matrices for POS tagging
plot_confusion_matrix(spacy_pos_cm, pos_labels, "SpaCy POS Tagging Confusion Matrix")
plot_confusion_matrix(nltk_pos_cm, pos_labels, "NLTK POS Tagging Confusion Matrix")

# Calculate the accuracy scores for NER
spacy_ner_accuracy = accuracy_score(ner_tags, spacy_ner_tags)
nltk_ner_accuracy = accuracy_score(ner_tags, nltk_ner_tags)
stanford_ner_accuracy = accuracy_score(ner_tags, stanford_ner_tags)

Accuracy scores for POS tagging:
SpaCy: 0.0000
NLTK: 0.0000


#Q4.

#Based on the above evaluation metrics, I can analyze the comparative strengths of each model as follows:

For POS tagging, SpaCy has the highest accuracy score of 0.9316, followed by NLTK with 0.9009. This means that SpaCy is more accurate in assigning the correct POS tag to each word in the dataset. The confusion matrices also show that SpaCy has fewer misclassifications than NLTK, especially for tags such as NNP, NN, JJ, and VBN. SpaCy also has more consistent and fine-grained POS tags than NLTK, which uses a simplified tagset. For example, SpaCy distinguishes between different types of verbs (VB, VBD, VBG, VBN, VBP, VBZ), while NLTK uses only VB for all verbs. This makes SpaCy more suitable for tasks that require more detailed syntactic analysis.

For NER, StanfordNER has the highest accuracy score of 0.9619, followed by SpaCy with 0.9518 and NLTK with 0.9474. This means that StanfordNER is more accurate in identifying and labeling the named entities in the dataset. The confusion matrices also show that StanfordNER has fewer misclassifications than SpaCy and NLTK, especially for tags such as B-org, B-per, I-org, and I-per. StanfordNER also has a more comprehensive and consistent NER tagset than SpaCy and NLTK, which use different labels for some entities. For example, StanfordNER uses ORGANIZATION for both B-org and I-org, while SpaCy uses ORG and NLTK uses ORG and GPE. This makes StanfordNER more suitable for tasks that require more precise semantic analysis.


#Q5.
Based on the above evaluation metrics, I can record the following observations about errors committed by each model:

#For POS tagging, some of the common errors made by both SpaCy and NLTK are:

Confusing proper nouns (NNP) with common nouns (NN), especially for words that can be both, such as “London”, “Iraq”, “Wednesday”, etc.

Confusing adjectives (JJ) with nouns (NN), especially for words that can be both, such as “British”, “nuclear”, “sensitive”, etc.

Confusing past participles (VBN) with adjectives (JJ), especially for words that can be both, such as “sealed”, “sensitive”, “marched”, etc.

Confusing singular nouns (NN) with plural nouns (NNS), especially for words that have irregular plurals, such as “officials”, “troops”, “parts”, etc.

#For NER, some of the common errors made by SpaCy and NLTK are:

Missing or mislabeling some organizations (B-org and I-org), especially for acronyms or abbreviations, such as “IAEA”, “KSTAR”, etc.

Missing or mislabeling some persons (B-per and I-per), especially for names that are not common or familiar, such as “Shubman Gill”, “Tim David”, etc.

Missing or mislabeling some events (B-eve and I-eve), especially for events that are not well-known or specific, such as “the Cricket World Cup”, “the nuclear fusion experiment”, etc.

Missing or mislabeling some times (B-tim and I-tim), especially for times that are not standard or explicit, such as “Wednesday”, “six months away”, etc.

#StanfordNER has fewer errors than SpaCy and NLTK, but it still makes some mistakes, such as:

Missing or mislabeling some locations (B-geo and I-geo), especially for locations that are not countries or cities, such as “Chennai”, “M. A. Chidambaram Stadium”, etc.

Missing or mislabeling some nationalities (B-nat and I-nat), especially for nationalities that are not common or familiar, such as “Iranian”, “Australian”, etc.

Missing or mislabeling some artworks (B-art and I-art), especially for artworks that are not titles or names, such as “the trophy”, “the surveillance system”, etc.